In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium.webdriver.common.keys import Keys
import html5lib
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException

In [78]:
result = {}
driver = webdriver.Chrome('chromedriver')
number = 1
    #사이트 접속 page 1~70 페이지당 20개 마지막 페이지 15개
for page in tqdm(range(1,71)):
    content =[]
    driver.get('http://www.saramin.co.kr/zf_user/public-recruit/coverletter-list/page/%d?coverletter_item=75'%page)
    time.sleep(0.5)
    for t in range(1, 21):
        content =[]
        #항목 클릭
        driver.find_element_by_xpath('//*[@id="listArea"]/table/tbody/tr[%d]/td[2]/h2/a'%t).click()
        time.sleep(1)
        #내용 가져오기 
        #1.첫문항에 '동기'가 있을 경우 -> 해당항목 글을 가져오고 뒤로
        #2. 없을경우 -> 항목 제목에서 '동기'을 찾아서 펼쳐보기 후 글을 가져옴
        text =""    
        text = driver.find_element_by_class_name('box_ty3').text
        
        if text.find('동기' or '지원') !=-1:
            text =text.replace('\n','')
            content.append(text)
            result[number]= content
            number += 1
            driver.get('http://www.saramin.co.kr/zf_user/public-recruit/coverletter-list/page/%d?coverletter_item=75'%page)
            time.sleep(1)
        else:
            x= driver.find_elements_by_class_name('item_self')
            
            for j in range(1, len(x)):
                q = driver.find_elements_by_class_name('item_self')[j].text

                if q.find('동기' or '지원') != -1:
                    #펼쳐보기 클릭
                    driver.find_elements_by_class_name('btn_tsp_show')[j].click()
                    time.sleep(1)
                    text = driver.find_elements_by_class_name('box_ty3')[j].text
                    text =text.replace('\n','')
                    content.append(text)
                    result[number]= content
                    number += 1
                    break

            driver.get('http://www.saramin.co.kr/zf_user/public-recruit/coverletter-list/page/%d?coverletter_item=75'%page)
            time.sleep(0.5)    
    

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="listArea"]/table/tbody/tr[16]/td[2]/h2/a"}
  (Session info: chrome=84.0.4147.105)


In [80]:
df = pd.DataFrame(result.items(), columns= ['id', 'content'])
df.to_csv('CVdata_raw.csv', index= False, encoding = "UTF-8-sig")

In [79]:
len(result)

1270